<a href="https://colab.research.google.com/github/24p11/recode-scenario/blob/main/generation_usable_icd_index_entries.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import ast
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
PATH_DATA = "/content/drive/MyDrive/Data/datasets/cepidc/original/"
PATH_ICD = "/content/drive/MyDrive/Data/referentiels/icd/"
PATH_RES ="/content/drive/MyDrive/Data/generation/index_cim/gen_index_20251109/"

In [ ]:
df_icd = pd.read_csv(PATH_ICD +"cim_10_atih_2019.tsv", sep="\t",header=None,names=["code","aut_mco","pos","aut_ssr","lib_court","libelle"])
df_icd.code = df_icd.code.str.replace(" ","")

#import unicodedata
#df.StandardText =  df.StandardText.map(lambda x: unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf8'))
#df.RawText =  df.RawText.map(lambda x: unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf8'))
#df_icd.libelle =  df_icd.libelle.map(lambda x: unicodedata.normalize('NFKD', x).encode('ascii', 'ignore').decode('utf8'))

In [ ]:
dataset_names = [PATH_DATA +"AlignedCauses_2006-2012full.csv",
                 PATH_DATA +"AlignedCauses_2013full.csv",
                 PATH_DATA +"AlignedCauses_2014_full.csv"]

df_cepidc =pd.concat(map(lambda file: pd.read_csv(file, sep=";"),
              dataset_names)).rename(columns={"YearCoded":"year",
                                              "Gender":"sex",
                                              "Age":"age"})

In [ ]:
df_cepidc.loc[df_cepidc.ICD10.isna(),"ICD10"]  = "NOCODE"
df_cepidc.loc[df_cepidc.StandardText.isna(),"StandardText"]  = "Pas code CIM codable"
df_cepidc = df_cepidc.merge(df_icd[["code","libelle"]], left_on="ICD10", right_on="code", how="left")

In [ ]:
PATH_DATA = "/content/drive/MyDrive/Projets/IA_Générative_college_DIM_2023/Groupe de travail IA codage/AOC/data/cim_mistral_synomyns.xlsx"
df_mistral_syn= pd.read_excel(PATH_DATA,index_col=0)

In [ ]:
df_hector_1= pd.read_excel(PATH_ICD + "Dictionnaire_Hector_MAJ062019.xlsx",sheet_name="Cim Analytique",names =["libelle","source","code","autre_code"])
df_hector_2= pd.read_excel(PATH_ICD + "Dictionnaire_Hector_MAJ062019.xlsx",sheet_name="Cim Alphabétique",names =["libelle","source","code","autre_code"])
df_hector = pd.concat([df_hector_1,df_hector_2],axis =0)
tmp= pd.read_excel(PATH_ICD + "Dictionnaire_Hector_MAJ062019.xlsx",sheet_name="Thesam",names =["libelle","source","code","autre_code"])
df_hector = pd.concat([df_hector,tmp],axis =0)
tmp= pd.read_excel(PATH_ICD + "Dictionnaire_Hector_MAJ062019.xlsx",sheet_name="Dermatologie",names =["libelle","source","code","autre_code"])
df_hector = pd.concat([df_hector,tmp],axis =0)
tmp= pd.read_excel(PATH_ICD + "Dictionnaire_Hector_MAJ062019.xlsx",sheet_name="Endocrinologie",names =["libelle","source","code","autre_code"])
df_hector = pd.concat([df_hector,tmp],axis =0)
tmp= pd.read_excel(PATH_ICD + "Dictionnaire_Hector_MAJ062019.xlsx",sheet_name="GRONES",names =["libelle","source","code","autre_code"])
df_hector = pd.concat([df_hector,tmp],axis =0)
tmp= pd.read_excel(PATH_ICD + "Dictionnaire_Hector_MAJ062019.xlsx",sheet_name="Troubles métaboliques",names =["libelle","source","code","autre_code"])
df_hector = pd.concat([df_hector,tmp],axis =0)
tmp= pd.read_excel(PATH_ICD + "Dictionnaire_Hector_MAJ062019.xlsx",sheet_name="Néphrologie",names =["libelle","source","code","autre_code"])
df_hector = pd.concat([df_hector,tmp],axis =0)
tmp= pd.read_excel(PATH_ICD + "Dictionnaire_Hector_MAJ062019.xlsx",sheet_name="Ophtalmo",names =["libelle","source","code","autre_code"])
df_hector = pd.concat([df_hector,tmp],axis =0)
tmp= pd.read_excel(PATH_ICD + "Dictionnaire_Hector_MAJ062019.xlsx",sheet_name="Orphanet",names =["libelle","source","code","autre_code"])
df_hector = pd.concat([df_hector,tmp],axis =0)
tmp= pd.read_excel(PATH_ICD + "Dictionnaire_Hector_MAJ062019.xlsx",sheet_name="Rhumatologie",names =["libelle","source","code","autre_code"])
df_hector = pd.concat([df_hector,tmp],axis =0)
tmp= pd.read_excel(PATH_ICD + "Dictionnaire_Hector_MAJ062019.xlsx",sheet_name="Germes",names =["libelle","source","code","autre_code"])
df_hector = pd.concat([df_hector,tmp],axis =0)
tmp= pd.read_excel(PATH_ICD + "Dictionnaire_Hector_MAJ062019.xlsx",sheet_name="SRLF",names =["libelle","source","code","autre_code"])
df_hector = pd.concat([df_hector,tmp],axis =0)

In [ ]:
df_hector.groupby("source").size()

,0
source,
A,40303
B,45266
DR1,1834
ED1,301
GRONES,166
METABOL,269
NP1,715
OP1,444
ORPHA,10274


In [ ]:
df_index = df_hector[ (df_hector["source"]=="B") ].rename(columns={"libelle":"index_cim"})
df_index = df_index.merge(df_icd[["code","libelle"]], on="code", how="left")

In [ ]:
#!pip install mistralai

In [ ]:
import requests
import time
import logging
from tenacity import retry, stop_after_attempt, wait_exponential


In [ ]:
import os
from mistralai import Mistral

In [ ]:
api_url = "https://api.mistral.ai/v1/chat"
api_key = "j17liFFXlV5X54DDPdvax0hxHBHXuSa9"
model = "mistral-large-latest"

In [ ]:
client = Mistral(api_key=api_key)

In [ ]:
from io import BytesIO
from mistralai import File
import json
import time

In [ ]:
system_prompt = """
Tu es un assistant spécialisé dans la reformulation des entrées de l'**index alphabétique de la CIM-10** pour faciliter la recherche de mots-clés dans des comptes rendus médicaux.

#### **Contexte :**
La Classification Internationale des Maladies (CIM-10) est un système de codage médical standardisé, recommandé par l’OMS, utilisé pour :
- Classer les diagnostics médicaux pour la mortatlié et la moribidité.
- Faciliter la facturation (ex. : T2A en France).
- Produire des statistiques de santé publique.
Hiérarchie de la CIM-10 :
- 21 chapitres (ex. : Chapitre II pour les tumeurs).
- Blocs : Regroupements homogènes de catégories à 3 caractères (ex. : "C00-C97" pour les tumeurs malignes).
- Catégories à 3 caractères (ex. : "C15" pour les tumeurs malignes de l’œsophage).
- Codes à 4 caractères (ex. : "C15.9" pour une tumeur maligne de l’œsophage, sans précision).
Contenu de la classification :
- Tableau analitique : liste hiérachisée des codes et de leur organisation en chapitre, bloc, catérogie et code
- Index alphabétique : liste des termes facilitant la recherche de codes pour les codeurs.


Les entrées de l'index alphabétique de la CIM-10 sont rédigées selon des règles spécifiques à la science des classificaitions en vigeur lors de sa publication. En pratique ces entrées sont parfois formulé de telle façon que le texte lui même ne peut pas tel quel être retrouvé dans un document médical rédigés par un médecin dans le cadre du soins (document clinique de type compte redu de consultation ou d'hopsitalisation).
Ton travail sera de proposer des reformulations d'entrées de l'indeux de la CIM-10 en termes décrivant le diagnostic qui soit succeptibles d'être retrouvés dans un compte rendu médical.
Les données suivantes te seront fournies :
- **Code CIM-10** : [CODE]
- **Libellé officiel** : "[LIBELLÉ OFFICIEL]"
- **Entrée officielle de l'index** : "[ENTREE OFFICIELLE INDEX]"

#### **Consignes strictes :**
1. **Objectif** :
   Générer **toutes les combinaisons textuelles possibles** à partir de l'entrée officielle de l'index lorsque c'est nécessaire :
   - Des mots entre parenthèses (optionnels).
   - Des formulations inversées (ex. : "maladie d'Eberth" ↔ "Eberth, maladie d'").
   - Des prépositions et apostrophes (ex. : "à", "de", "du").
   Attention toutes les entrées de l'index seront traitées séparément, donc des entrées ne nécessitant pas de transformation te se seront pafois fournies, ce sont celles qui sont suceptibles d'être retrouvées telles quelles dans des documents cliniques.
   Les entrées alternatives que tu proposera doivent pouvoir être utilisées telles quelles dans des documents médicaux. Mais attention, elles ne doivent comporter que des mots de l'entrée de l'index.

2. **Règles à appliquer** :
   - **Mots entre parenthèses** : Optionnels. Générer toutes les combinaisons en les incluant ou excluant.
   - **Formulations inversées** :
     - Si l'entrée est de la forme `"[X], maladie de"`, générer aussi `"maladie de [X]"`.
     - Exemple : `"Eberth, maladie d'"` → `"maladie d'Eberth"`.
   - **Prépositions et apostrophes** :
     - Respecter les contractions (ex. : `"d'"` au lieu de `"de"` si le mot suivant commence par une voyelle).
     - Tester les variantes avec/sans prépositions (ex. : `"infection à mycobacterium"` et `"infection mycobacterium"`).
   - **Ordre des mots** :
     - Pour les adjectifs ou compléments, tester les permutations logiques (ex. : `"infection atypique à mycobacterium"` et `"infection à mycobacterium atypique"`).
   - **Sans précision** :   Les entrées proposées doivent être suceptibles d'être retrouvés dans les document médicaux, les termes du vocabulaire spécifique de la CIM10 comme sans précision doivent être supprimés.
   - **Cohérence avec le libellé officiel** :
     - Les combinaisons doivent **refléter la définition médicale** du code CIM-10 fourni.
     - Exclure toute combinaison qui s'éloignerait du sens du libellé.
   - **Pas de mots externes** : Se limiter strictement aux mots de l'entrée.
   - **Entrées à ne pas changer** : Les entrées qui sont des expressions suceptibles d'être utilisée telles quelles dans des documents médicaux ne doivent pas être modifiées.

3. **Format de sortie** :
   - Retourner un objet JSON avec :
     - `"code"` : Le code CIM-10.
     - `"libellé"` : Le libellé officiel.
     - `"entrée"` : L'entrée de l'index.
     - `"entrées proposées"` : Liste exhaustive des combinaisons valides.

4. **Exemples traités** :
   - **Entrée** : `"Infection (à) (de), mycobacterium, mycobactérienne, atypique"`
     **Code** : `A319`
     **Libellé** : `"Infection à mycobactéries atypiques, sans précision"`
     Résultat :
     ```json
     {
       "code": "A319",
       "libellé": "Infection à mycobactéries atypiques, sans précision",
       "entrée": "Infection (à) (de), mycobacterium, mycobactérienne, atypique",
       "combinations": [
         "infection à mycobacterium atypique",
         "infection de mycobacterium atypique",
         "infection mycobactérienne atypique",
         "infection atypique à mycobacterium",
         "infection atypique mycobactérienne"
       ]
     }
     ```
   - **Entrée** : `"Eberth, maladie d'"`
     **Code** : `A02.0`
     **Libellé** : `"Fièvre typhoïde"`
     Résultat :
     ```json
     {
       "code": "A02.0",
       "libellé": "Fièvre typhoïde",
       "entrée": "Eberth, maladie d'",
       "combinations": [
         "maladie d'Eberth",
         "Eberth, maladie d'"
       ]
     }
     ```




 """

In [ ]:
df_index['system'] = system_prompt
df_index['user'] = df_index.apply(lambda x: "Entrée à traiter : -**Code CIM-10**: " + x.code +  " -** Libellé officiel** : " + str(x.libelle) +"- **Entrée de l'index** : " + x.index_cim, axis=1)


In [ ]:
import argparse
import json
import os
import random
import time
from io import BytesIO

import httpx
from mistralai import File, Mistral


def create_client():
    """
    Create a Mistral client using the API key from environment variables.

    Returns:
        Mistral: An instance of the Mistral client.
    """
    return Mistral(api_key=os.environ["MISTRAL_API_KEY"])

def generate_random_string(start, end):
    """
    Generate a random string of variable length.

    Args:
        start (int): Minimum length of the string.
        end (int): Maximum length of the string.

    Returns:
        str: A randomly generated string.
    """
    length = random.randrange(start, end)
    return ' '.join(random.choices('abcdefghijklmnopqrstuvwxyz', k=length))

def print_stats(batch_job):
    """
    Print the statistics of the batch job.

    Args:
        batch_job: The batch job object containing job statistics.
    """
    print(f"Total requests: {batch_job.total_requests}")
    print(f"Failed requests: {batch_job.failed_requests}")
    print(f"Successful requests: {batch_job.succeeded_requests}")
    print(
        f"Percent done: {round((batch_job.succeeded_requests + batch_job.failed_requests) / batch_job.total_requests, 4) * 100}")


def create_input_file(client, data):
    """
    Create an input file for the batch job.

    Args:
        client (Mistral): The Mistral client instance.
        num_samples (int): Number of samples to generate.

    Returns:
        File: The uploaded input file object.
    """
    buffer = BytesIO()
    for idx in range(len(data)):
        request = {
            "custom_id": str(idx),
            "body": {
                "max_tokens": random.randint(10, 1000),
                "messages": [
          {
                          "role": "system",
                          "content": data.system[idx]
                          },
                      {
                          "role": "user",
                          "content": data.user[idx]
                          }
      ]
            }
        }
        buffer.write(json.dumps(request).encode("utf-8"))
        buffer.write("\n".encode("utf-8"))
    return client.files.upload(file=File(file_name="file.jsonl", content=buffer.getvalue()), purpose="batch")


def run_batch_job(client, input_file, model):
    """
    Run a batch job using the provided input file and model.

    Args:
        client (Mistral): The Mistral client instance.
        input_file (File): The input file object.
        model (str): The model to use for the batch job.

    Returns:
        BatchJob: The completed batch job object.
    """
    batch_job = client.batch.jobs.create(
        input_files=[input_file.id],
        model=model,
        endpoint="/v1/chat/completions",
        metadata={"job_type": "testing"}
    )

    while batch_job.status in ["QUEUED", "RUNNING"]:
        batch_job = client.batch.jobs.get(job_id=batch_job.id)
        #print_stats(batch_job)
        time.sleep(1)

    print(f"Batch job {batch_job.id} completed with status: {batch_job.status}")
    return batch_job


def download_file(client, file_id, output_path):
    """
    Download a file from the Mistral server.

    Args:
        client (Mistral): The Mistral client instance.
        file_id (str): The ID of the file to download.
        output_path (str): The path where the file will be saved.
    """
    if file_id is not None:
        print(f"Downloading file to {output_path}")
        output_file = client.files.download(file_id=file_id)
        with open(output_path, "wb") as f:
            for chunk in output_file.stream:
                f.write(chunk)
        print(f"Downloaded file to {output_path}")

In [ ]:
nb_samples = 5000
print(len(df_index))
int(len(df_index)/nb_samples)

45266


9

In [ ]:
nb_samples = 5000


for i in range(1, int(len(df_index)/nb_samples)):
  print(str(i*nb_samples) + " - "+ str((i+1)*nb_samples) )
  prompts = df_index[["code","libelle","system", "user"]][i*nb_samples:(i+1)*nb_samples].reset_index(drop=True)
  input_file = create_input_file(client, prompts)
  print(f"Created input file {input_file}")
  batch_job = run_batch_job(client, input_file, model)
  print(f"Job duration: {batch_job.completed_at - batch_job.created_at} seconds")
  download_file(client, batch_job.output_file, PATH_RES + "batch_" +  str(i) + ".json" )


5000 - 10000
Created input file id='ba2795c5-a808-4719-a335-b0ac3c9dbe5f' object='file' size_bytes=33745205 created_at=1762703185 filename='file.jsonl' purpose='batch' sample_type='batch_request' source='upload' num_lines=5000 mimetype='application/jsonl' signature='ccb3f90a35cea7f5'
Batch job 2d87c18d-4649-4c6d-9cf9-bd36e763a948 completed with status: SUCCESS
Job duration: 315 seconds
Downloaded file to /content/drive/MyDrive/Data/generation/index_cim/gen_index_20251109/batch_1.json
10000 - 15000
Created input file id='f61d0a2f-01ec-4009-a13f-91bb950c57bc' object='file' size_bytes=33691912 created_at=1762703505 filename='file.jsonl' purpose='batch' sample_type='batch_request' source='upload' num_lines=5000 mimetype='application/jsonl' signature='044871770e9fe404'
Batch job e54268c0-1af2-4cd2-b38e-e0db870baeaf completed with status: SUCCESS
Job duration: 316 seconds
Downloaded file to /content/drive/MyDrive/Data/generation/index_cim/gen_index_20251109/batch_2.json
15000 - 20000
Created

In [ ]:
import json
import re
import pandas as pd

def extract_json_from_content(content: str) -> dict:
    """
    Extrait le JSON du contenu de la réponse Mistral.
    Le contenu est de la forme : ```json\n{...}\n```.
    """
    # Utiliser une expression régulière pour extraire le JSON
    match = re.search(r'```json\n(.*?)\n```', content, re.DOTALL)
    if match:
        json_str = match.group(1)
        try:
            return json.loads(json_str)
        except json.JSONDecodeError as e:
            print(f"Erreur de décodage JSON : {e}")
            return None
    return None

def process_mistral_batch_json(input_jsonl_path: str) -> pd.DataFrame:
    """
    Lit un fichier JSON Lines de sortie de l'API Mistral (format batch),
    extrait les combinaisons et génère un DataFrame structuré.
    """
    structured_data = []

    with open(input_jsonl_path, "r", encoding="utf-8") as f:
        for line in f:
            try:
                item = json.loads(line)
                # Accéder au contenu de la réponse
                response_body = item["response"]["body"]
                choices = response_body["choices"]

                if choices and len(choices) > 0:
                    content = choices[0]["message"]["content"]
                    parsed_json = extract_json_from_content(content)

                    if parsed_json and "entrées proposées" in parsed_json:  # Notez le nom de la clé
                        for combo in parsed_json["entrées proposées"]:
                            structured_data.append({
                                "code": parsed_json["code"],
                                "libellé": parsed_json["libellé"],
                                "entrée_originale": parsed_json["entrée"],
                                "combinaison": combo
                            })
            except json.JSONDecodeError as e:
                print(f"Erreur de décodage JSON sur la ligne : {line.strip()} - {e}")
                continue # Skip to the next line if there's an error

    # Création du DataFrame final
    df_output = pd.DataFrame(structured_data)

    return df_output

In [ ]:
import os
import json
df_final_index = pd.DataFrame()
output_dir = PATH_RES
# Assuming nb_samples is defined and correctly used to determine the number of batch files
num_batches = int(len(df_index) / nb_samples) + 1

for i in range(num_batches): # Iterate through all batch files
    file_path = os.path.join(output_dir, f"batch_{i}.json")
    if os.path.exists(file_path):
      # input_json_path = "sorties_brutes_mistral_batch.json"  # Remplacez par votre fichier
      # output_csv_path = "reformulations_structurées.csv" # No longer saving to CSV in this function

      df_result = process_mistral_batch_json(file_path)
      df_final_index = pd.concat([df_final_index,df_result],axis=0)



Erreur de décodage JSON : Expecting value: line 7 column 37 (char 192)
Erreur de décodage JSON : Expecting value: line 7 column 29 (char 178)
Erreur de décodage JSON : Expecting value: line 10 column 5 (char 268)
Erreur de décodage JSON : Expecting ',' delimiter: line 9 column 26 (char 226)
Erreur de décodage JSON : Expecting value: line 17 column 5 (char 561)
Erreur de décodage JSON : Expecting value: line 13 column 31 (char 477)
Erreur de décodage JSON : Expecting value: line 8 column 38 (char 226)
Erreur de décodage JSON : Expecting ',' delimiter: line 16 column 35 (char 633)
Erreur de décodage JSON : Expecting value: line 20 column 34 (char 500)
Erreur de décodage JSON : Expecting value: line 27 column 27 (char 1347)
Erreur de décodage JSON : Expecting value: line 13 column 5 (char 394)
Erreur de décodage JSON : Expecting ',' delimiter: line 7 column 22 (char 146)
Erreur de décodage JSON : Expecting ',' delimiter: line 8 column 28 (char 173)
Erreur de décodage JSON : Expecting valu

In [ ]:
df_final_index

,code,libellé,entrée_originale,combinaison
0,A163,"Tuberculose des ganglions intrathoraciques, (s...","Adénite (de), médiastinale, tuberculeuse (avec...",adénite médiastinale tuberculeuse
1,A163,"Tuberculose des ganglions intrathoraciques, (s...","Adénite (de), médiastinale, tuberculeuse (avec...",adénite tuberculeuse médiastinale
2,A163,"Tuberculose des ganglions intrathoraciques, (s...","Adénite (de), médiastinale, tuberculeuse (avec...",adénite médiastinale tuberculeuse avec manifes...
3,A163,"Tuberculose des ganglions intrathoraciques, (s...","Adénite (de), médiastinale, tuberculeuse (avec...",adénite tuberculeuse médiastinale avec manifes...
4,A163,"Tuberculose des ganglions intrathoraciques, (s...","Adénite (de), médiastinale, tuberculeuse (avec...",adénite de médiastin tuberculeuse
...,...,...,...,...
38016,Z907,Absence acquise d'organe(s) génital(aux),"Absence (complète ou partielle) (de), vulve (c...",absence complète acquise de vulve
38017,Z907,Absence acquise d'organe(s) génital(aux),"Absence (complète ou partielle) (de), vulve (c...",absence partielle acquise de la vulve
38018,Z907,Absence acquise d'organe(s) génital(aux),"Absence (complète ou partielle) (de), vulve (c...",absence partielle acquise de vulve
38019,Z907,Absence acquise d'organe(s) génital(aux),Hystérectomisée,hystérectomisée


In [ ]:
df_cepidc

,DocID,year,sex,age,LocationOfDeath,LineID,RawText,IntType,IntValue,CauseRank,StandardText,ICD10,code,libelle
0,1,2006,1,20,1.0,1,Tableau de mort subite de cause inconnue au te...,NaN,NaN,NaN,Pas code CIM codable,NOCODE,NaN,NaN
1,2,2006,1,30,5.0,1,Carbonisation diffuse avec traumatisme thoracique,NaN,NaN,1-1,carbonisation,T293,T293,"Brûlures de parties multiples du corps, au moi..."
2,2,2006,1,30,5.0,1,Carbonisation diffuse avec traumatisme thoracique,NaN,NaN,1-2,traumatisme thoracique,S299,S299,"Lésion traumatique du thorax, sans précision"
3,2,2006,1,30,5.0,2,Accident de la voie publique (victime conducte...,NaN,NaN,2-1,accident voie publique,V892,NaN,NaN
4,2,2006,1,30,5.0,5,hypertrophie ventriculaire gauche,NaN,NaN,6-1,hypertrophie ventricule gauche,I517,I517,Cardiomégalie
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509097,139723,2014,1,90,2.0,1,SYNDROME OCCLUSIF,NaN,NaN,1-1,syndrome occlusif,K566,K566,"Occlusions intestinales, autres et sans précision"
509098,139723,2014,1,90,2.0,2,ADENOCARCINOME COLIQUE,NaN,NaN,2-1,adénocarcinome colique,C189,C189,"Tumeur maligne du côlon, sans précision"
509099,127638,2014,2,85,2.0,1,GRABATAIRE,6,0,1-1,grabataire,R263,R263,Immobilité
509100,127638,2014,2,85,2.0,2,DEMENCE DE TYPE ALZHEIMER EVOLUEE,6,0,2-1,démence type alzheimer évoluée,G309,G309,"Maladie d'Alzheimer, sans précision"


In [ ]:
PATH_FINAL = "/content/drive/MyDrive/Projets/IA_Générative_college_DIM_2023/Groupe de travail IA codage/AOC/data/cim_index_modifie.csv"

In [ ]:
df_final_index.rename(columns={"libellé":"icd_description",	"entrée_originale" : "index_orginal", 	"combinaison": "index_reformulate"}).to_csv(PATH_FINAL, sep=";",index=False)

In [ ]:
df_final_index = df_final_index[~(df_final_index.code.str.contains("TERMINER"))]


In [ ]:
df_final_index[df_final_index.code.str.contains("Z")]

,code,libellé,entrée_originale,combinaison
26225,Z003,Examen de l'adolescent en cours de croissance,"Surveillance (médicale) (de) (pour), développe...",surveillance médicale du développement à l'ado...
26226,Z003,Examen de l'adolescent en cours de croissance,"Surveillance (médicale) (de) (pour), développe...",surveillance médicale pour le développement à ...
26227,Z003,Examen de l'adolescent en cours de croissance,"Surveillance (médicale) (de) (pour), développe...",surveillance du développement à l'adolescence
26228,Z003,Examen de l'adolescent en cours de croissance,"Surveillance (médicale) (de) (pour), développe...",surveillance pour le développement à l'adolesc...
26229,Z003,Examen de l'adolescent en cours de croissance,"Surveillance (médicale) (de) (pour), développe...",surveillance médicale du développement en adol...
...,...,...,...,...
38016,Z907,Absence acquise d'organe(s) génital(aux),"Absence (complète ou partielle) (de), vulve (c...",absence complète acquise de vulve
38017,Z907,Absence acquise d'organe(s) génital(aux),"Absence (complète ou partielle) (de), vulve (c...",absence partielle acquise de la vulve
38018,Z907,Absence acquise d'organe(s) génital(aux),"Absence (complète ou partielle) (de), vulve (c...",absence partielle acquise de vulve
38019,Z907,Absence acquise d'organe(s) génital(aux),Hystérectomisée,hystérectomisée


In [ ]:
df_cepidc[df_cepidc.code.str.contains("R")]

,DocID,year,sex,age,LocationOfDeath,LineID,RawText,IntType,IntValue,CauseRank,StandardText,ICD10,code,libelle
5,3,2006,1,10,1.0,1,Tableau de mort subite au cours d'un effort sp...,NaN,NaN,1-1,mort subite,R960,R960,Mort instantanée
9,4,2007,2,80,3.0,1,arrêt cardio-respiratoire hypoxémique,1,15,1-1,arrêt cardio-respiratoire,R092,R092,Arrêt respiratoire
10,4,2007,2,80,3.0,1,arrêt cardio-respiratoire hypoxémique,1,15,1-2,hypoxémie,R090,R090,Asphyxie
21,6,2007,1,50,1.0,1,Possible décompensation d'une cardiopathie isc...,NaN,NaN,1-2,cadavre putréfié,R98,R98,Décès sans témoin
26,7,2007,1,95,3.0,5,"polyarthrose invalidante, altération des fonc...",NaN,NaN,6-2,altération fonctions cognitives,R418,R418,Symptômes et signes relatifs aux fonctions cog...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
509069,110031,2014,2,80,4.0,1,Arrêt des fonctions vitales entrant dans le ca...,3,21,1-2,fin de vie,R688,R688,Autres symptômes et signes généraux précisés
509072,110031,2014,2,80,4.0,2,Démence de type vasculaire au stade de la grab...,5,6,2-5,troubles mnésiques,R413,R413,Autres formes d'amnésie
509074,110031,2014,2,80,4.0,2,Démence de type vasculaire au stade de la grab...,5,6,2-7,apraxie,R482,R482,Apraxie
509077,110031,2014,2,80,4.0,2,Démence de type vasculaire au stade de la grab...,5,6,2-8,aphasie,R470,R470,Dysphasie et aphasie


In [ ]:
df_prep = df_hector[~ df_hector.source.isin(["A","B"])].rename(columns={"libelle":"extrait"}).merge(df_icd[["code","libelle"]].rename(columns={"libelle":"definition"}) )[["extrait","code","definition"]]

In [ ]:
df_prep = pd.concat([df_prep,df_final_index[["combinaison","code","libellé"]].rename(columns={"libellé":"definition",		"combinaison": "extrait"})])

In [ ]:
import random

# Générer des exemples négatifs (1 négatif par positif)
def generate_negative_examples(data):
    negative_examples = []
    codes = [d["code"] for d in data] # Create a list of all codes for sampling

    for example in data:
        if example["label"] == 1:
            example_code = example["code"]

            # Determine the type of negative example based on probabilities
            choice = random.random()

            if choice < 0.5:  # 50% same first 3 characters
                possible_codes = [code for code in codes if code[:3] == example_code[:3] and code != example_code]
            elif choice < 0.8: # 30% same first character
                possible_codes = [code for code in codes if code[0] == example_code[0] and code != example_code]
            else: # 20% different first character
                possible_codes = [code for code in codes if code[0] != example_code[0]]

            if possible_codes:
                negative_code = random.choice(possible_codes)
                # Find the corresponding definition for the negative code
                negative_definition = next((d["definition"] for d in data if d["code"] == negative_code), "Definition not found")

                negative_examples.append({
                    "extrait": example["extrait"],
                    "code": negative_code,
                    "definition": negative_definition,
                    "label": 0
                })
    return negative_examples